In [1]:
%%capture
!pip install bitsandbytes

!pip install sentencepiece

!pip install -qq -U transformers
!pip install -qq -U dspy-ai
!pip install -qq -U proto-plus
!pip install -qq -U dspy-ai[faiss-cpu]



**DSPy is a framework that simplifies and automates the process of optimizing language model (LM) prompts and weights, especially in complex pipelines. It separates program flow from prompt tuning and uses LM-driven algorithms to systematically improve performance, reducing the need for manual tweaking and enabling more reliable results.**

# What the code is doing (step-by-step):

* Loads vectorizer model: Prepares a sentence similarity model ("l3cube-pune/bengali-sentence-similarity-sbert") for Bangla.
* Loads dataset: Fetches a Bengali question-answer dataset from the csebuetnlp/squad_bn dataset and formats it into a DataFrame.
* Preprocesses the dataset: Cleans the dataset by filtering out invalid entries and resetting the index.
* Defines the evaluate_multilingual_model function: Handles the multilingual RAG pipeline setup and model evaluation.
* Sets up Faiss retrieval module: Initializes a Faiss-based retriever using the pre-defined vectorizer for searching through contexts.
* Configures language model: Loads and configures a generator model (unsloth/gemma-2-9b-it-bnb-4bit) to generate answers.
* Splits data: Splits the cleaned dataset into training and validation sets (train = 1063 and validation = 188).
* Defines model classes: Implements the GenerateAnswer class(it works with dspy.Signature) for answering questions and the BanglaRAG class for RAG processing.
* Implements answer validation: Sets up a function to validate context and generated answers using exact match metrics.
* Creates RAG model: Initializes the BanglaRAG pipeline for retrieval and generation.
* Bootstraps the model: Uses DSPy's BootstrapFewShot optimizer to train the RAG model with few-shot learning.
* Compiles the model with a teleprompter: The teleprompter.compile method takes the RAG model and the training dataset and optimizes it using a few-shot learning approach. It fine-tunes the model by selecting and using the most effective prompts (based on a metric) to generate more accurate results for each input. The process helps in building a pipeline where prompts and responses are improved automatically based on the examples provided.
* Evaluates the model: Runs evaluation on the validation set, displaying progress and measuring performance using exact match metrics. We got Average Metric: 44.0 / 188  (23.4)

# References : 
1. https://arxiv.org/abs/2310.03714
2. https://github.com/stanfordnlp/dspy/issues/1479
   

In [2]:
%%time

import pandas as pd
import dspy
from dspy.retrieve import faiss_rm
from dsp.modules.sentence_vectorizer import SentenceTransformersVectorizer

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate
from dspy.evaluate import answer_exact_match
from dspy.retrieve.faiss_rm import FaissRM

vectorizer = SentenceTransformersVectorizer(model_name_or_path="l3cube-pune/bengali-sentence-similarity-sbert")

dataset_name="csebuetnlp/squad_bn"
ds = load_dataset(dataset_name, trust_remote_code=True)
df = pd.DataFrame(ds['validation'])
df['answers'] = df['answers'].apply(lambda x: x['text'][0] if x['text'] else '')
df = df[['question', 'answers', 'context']].dropna().query("question != '' and answers != '' and context != ''")
df = df.reset_index(drop=True)
#interesting problem for : BanglaLLM/BanglaLLama-3-8b-BnWiki-Instruct and BanglaLLM/bangla-llama-7b-base-v0.1
generator_model = "unsloth/gemma-2-9b-it-bnb-4bit"

def evaluate_multilingual_model(retriever_model="l3cube-pune/bengali-sentence-similarity-sbert", 
                                generator_model=generator_model, 
                                df=None,
                                debug_mode=False):
    
    if debug_mode:
        df = df.head(5)

    # class MultilingualVectorizer:
    #     def __init__(self, model_name=retriever_model):
    #         self.model = SentenceTransformer(model_name)

    #     def __call__(self, texts):
    #         return self.model.encode(texts, convert_to_tensor=False)
    
    # multilingual_vectorizer = MultilingualVectorizer()
    # frm = faiss_rm.FaissRM(df['context'].tolist(), vectorizer=multilingual_vectorizer)

    frm = FaissRM(df['context'],vectorizer = vectorizer)
    
    model = dspy.HFModel(model=generator_model)
    model.drop_prompt_from_output = True
    dspy.settings.configure(lm=model, rm=frm)

    
    dataset = [dspy.Example(question=q, answer=a).with_inputs("question") for q, a, c in df.values]
    train, val = train_test_split(dataset, test_size=0.15, random_state=42)
    print(len(train), len(val))
    class GenerateAnswer(dspy.Signature):
        """Answer questions with short factoid answers in bangla."""
    
        context = dspy.InputField(desc="may contain relevant facts")
        question = dspy.InputField()
        answer = dspy.OutputField(desc="often between 1 and 5 words")
    
    class BanglaRAG(dspy.Module):
        def __init__(self, num_passages=3):
            super().__init__()
            self.retrieve = dspy.Retrieve(k=num_passages)
            self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
            #self.generate_answer = dspy.Predict(GenerateAnswer)

        def forward(self, question):
            context = self.retrieve(question).passages
            prediction = self.generate_answer(context=context, question=question)
            return dspy.Prediction(context=context, answer=prediction.answer)
           

    def validate_context_and_answer(example, pred, trace=None):
        answer_EM = dspy.evaluate.answer_exact_match(example, pred)
        answer_PM = dspy.evaluate.answer_passage_match(example, pred)
        return answer_EM and answer_PM

    bangla_rag_model = BanglaRAG()
    # from dspy.teleprompt import MIPROv2
    # teleprompter = MIPROv2( metric=validate_context_and_answer, num_candidates=50, init_temperature=1.0)

    teleprompter = BootstrapFewShot(metric=validate_context_and_answer,max_labeled_demos=50)
    compiled_rag = teleprompter.compile(bangla_rag_model, trainset=train)
    
    evaluate_on_dataset = Evaluate(devset=val, num_threads=1, display_progress=True, display_table=25)
    metric = answer_exact_match
    evaluate_on_dataset(compiled_rag, metric=metric)
    
    # prediction = compiled_rag(train[2].question).answer
    # print(f"Prediction on 3rd sample: {prediction}")

    #print(model.inspect_history(n=3))


evaluate_multilingual_model(df=df,debug_mode=False)


/home/mobassir/miniconda3/envs/apsis_mobassir/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
WARNING clustering 1251 points to 100 centroids: please provide at least 3900 training points
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


1063 188


  0%|                                                                         | 0/1063 [00:00<?, ?it/s]/home/mobassir/miniconda3/envs/apsis_mobassir/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  2%|█▏                                                            | 21/1063 [01:51<1:32:28,  5.32s/it]


Bootstrapped 4 full traces after 22 examples in round 0.
Average Metric: 0 / 2  (0.0):   1%|▍                                   | 2/188 [00:12<18:39,  6.02s/it]

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


...

Average Metric: 38.0 / 174  (21.8):  93%|█████████████████████████▉  | 174/188 [31:59<02:53, 12.41s/it]

2024-09-11T19:51:30.139845Z [error    ] Error for example in dev set: 		 -1 [dspy.evaluate.evaluate] filename=evaluate.py lineno=183


Average Metric: 44.0 / 188  (23.4): 100%|████████████████████████████| 188/188 [34:31<00:00, 11.02s/it]


,question,example_answer,context,pred_answer,answer_exact_match,answer
0,বাংলাদেশের সরকারী তোলারাম কলেজের বর্তমান মোট শিক্ষার্থীর সংখ্যা কত ?,১৮০০০,"['১৯৮০ খ্রিস্টাব্দের ১ মার্চ এই কলেজটিকে জাতীয়করণ করা হয়। সরকারি কলেজের মুকুট পরিধান করে কলেজের সুনাম, কলেবর এবং কার্যক্রম ক্রমান্বয়ে বৃদ্ধি পেতে থাকে। তোলারাম কলেজের...",প্রায় ১৮০০০,False,nan
1,বিখ্যাত জ্যোতির্বিজ্ঞানী কোপার্নিকাসের জন্ম কোথায় হয় ?,১৪৭৩ সালের ১৮ ফেব্রুয়ারী,"['নিকোলাস কোপারনিকাস (পলিশ ভাষায় মিকলজ কোপারনিক , জার্মান ভাষায় নিক্লাস কপারনিক, ১৪৭৩ সালের ১৮ ফেব্রুয়ারী জন্ম\\ ২৪ মে ১৫৪৩ মৃত্যু) ছিলেন রেনেসাঁ এবং সংস্কার যুগের...",পলিশ,False,nan
2,সত্যেন্দ্রনাথ বসু কত সালে জন্মগ্রহণ করেন ?,১ জানুয়ারি ১৮৯৪,['সত্যেন্দ্রনাথ বসু (১ জানুয়ারি ১৮৯৪ – ৪ ফেব্রুয়ারি ১৯৭৪) ছিলেন একজন ভারতীয় বাঙালি পদার্থবিজ্ঞানী। তাঁর গবেষণার ক্ষেত্র ছিল গাণিতিক পদার্থবিদ্যা। সত্যেন্দ্রনাথ বসু আলবার্ট আইনস্টাইনের সঙ্গে...,১৮৯৪,False,nan
3,প্রগতি লেখক ও শিল্পী সংঘ উদীচী সাংস্কৃতিক সংগঠনের প্রতিষ্ঠাতা সত্যেন সেনের জন্ম কোথায় হয় ?,বিক্রমপুর (বর্তমান মুন্সীগঞ্জ জেলার) টঙ্গীবাড়ী উপজেলার সোনারং গ্রামের সেন পরিবারে,['সত্যেন সেন ১৯০৭ সালের মার্চ ২৮ তারিখে বিক্রমপুর (বর্তমান মুন্সীগঞ্জ জেলার) টঙ্গীবাড়ী উপজেলার সোনারং গ্রামের সেন পরিবারে জন্মগ্রহণ করেন। ছোটবেলায় ডাক নাম ছিল লস্কর। তার...,বিক্রমপুর (বর্তমান মুন্সীগঞ্জ জেলার) টঙ্গীবাড়ী উপজেলার সোনারং গ্রাম,False,nan
4,আয়তনের দিক থেকে জার্মানি ইউরোপের কততম বৃহত্তম রাষ্ট্র ?,৭ম,['জার্মানিতে নগরায়নের হার অত্যন্ত উঁচু। বার্লিন দেশের রাজধানী ও বৃহত্তম শহর। তবে প্রাক্তন পশ্চিম জার্মানির রাজধানী বন শহরে এখনও বেশ কিছু সরকারী অফিস রয়েছে। জার্মান...,The text doesn't provide information about Germany's ranking by size in Europe.,False,nan
5,বর্তমানে ঢাকা বিশ্ববিদ্যালয়ের মোট ছাত্রছাত্রীর সংখ্যা কত ?,"৩৮,০০০",['ঢাকা বিশ্ববিদ্যালয় বাংলাদেশের সবচেয়ে পুরানো বিশ্ববিদ্যালয়। এই বিশ্ববিদ্যালয় ১৯২১ সালে প্রতিষ্ঠিত হয়। বাংলাদেশের মোট উচ্চ শিক্ষিত জনগোষ্ঠির প্রায় ৭০%-ই এই বিশ্ববিদ্যালয়ের ছাত্র-ছাত্রী। প্রতিষ্ঠার পর থেকে...,"প্রায় ৩৮,০০০",False,nan
6,স্বামী বিবেকানন্দের শিষ্যা ভগিনী নিবেদিতা কবে প্রথম ভারতবর্ষে আসেন ?,১৮৯৮ সালে,"['ভগিনী নিবেদিতা (ইংরেজি: Sister Nivedita) (Bengali pronunciation:[sister niːbediːt̪aː] ) (জন্ম মার্গারেট এলিজাবেথ নোবেল\' (ইংরেজি: Margaret Elizabeth Noble)[1]; ২৮ অক্টোবর, ১৮৬৭ – ১৩ অক্টোবর, ১৯১১)[2][3] ছিলেন...",১৮৯৮,False,nan
7,একজন প্রাপ্ত বয়স্ক মানুষের মস্তিষ্কের গড় ওজন কত ?,১.৩৬ কেজি,"['মস্তিষ্ক হল কেন্দ্রীয় স্নায়ুতন্ত্রের একটি অপরিহার্য অংশ, যা করোটির অভ্যন্তরে অবস্থিত এবং দেহের প্রধান নিয়ন্ত্রণকেন্দ্র।ভ্রুণ অবস্থায় সুষুম্নাকান্ডের অগ্রবর্তী দন্ডাকার অংশ ভাঁজ হয়ে পর পর ৩টি...",১.৩৬ কেজি,✔️ [True],nan
8,মৌমাছি যেখানে মধু সংগ্রহ করে রাখে সেটিকে কী বলা হয় ?,nan,nan,nan,0.0,চাক
9,ইউরি আলেক্সেইভিচ্ গাগারিনের বাবার নাম কী ?,অ্যালেক্সে ইয়ানোভিচ গ্যাগারি,['গ্যাগারিন ১৯৩৪ সালের ৯ই মার্চ জাটস্কের/1} কাছে ক্লুসিনো গ্রামে (বর্তমানে রাশিয়ার মোলেনস্ক ওবলাস্ট) জন্মগ্রহণ করেন।[2] তার সম্মানে ১৯৬৮ সালে নিকটস্থ জাটস্ক শহরের নাম পরিবর্তন করে...,অ্যালেক্সে ইয়ানোভিচ গ্যাগারিন,False,nan


CPU times: user 55min 14s, sys: 6min 45s, total: 1h 2min
Wall time: 39min 17s


In [3]:
idx = 70
df.question[idx],df.answers[idx]

('আবু নয়ীম মোহাম্মদ মুনীর চৌধুরী কত সালে কবর নাটকটি রচনা করেছিলেন ?', '১৯৫৩')

In [4]:
df.context[idx]

'মুনীর চৌধুরী ১৯৫৩ সালে কারাবন্দী অবস্থায় কবর নাটকটি রচনা করেন। ১৯৫৮ সালে প্রকাশিত পূর্ববঙ্গ সরকারের ভাষা-সংস্কার কমিটির রিপোর্টের অবৈজ্ঞানিক ও সাম্প্রদায়িক বিষয়বস্তুর তীব্র সমালোচনা করে মুনীর চৌধুরী পূর্ববঙ্গের ভাষা কমিটির রিপোর্ট আলোচনা প্রসঙ্গে একটি দীর্ঘ ভাষাতাত্ত্বিক প্রবন্ধ লেখেন। ১৯৫৯ সালের ২৭শে এপ্রিল প্রবন্ধটি বাংলা একাডেমিতে পঠিত হয়। কিন্তু মুসলিম ধর্মবিশ্বাসে আঘাতের অভিযোগে সামরিক সরকারের কাছে তাঁকে কৈফিয়ৎ দিতে হয়। এরপর তিনি সাহিত্যে মনোনিবেশ করেন ও বেশ কিছু মৌলিক ও অনুবাদ নাটক লেখেন। অনেকগুলি প্রবন্ধের সংকলনও প্রকাশ করেন। মীর মানস (১৯৬৫) প্রবন্ধ সংকলনের জন্য দাউদ পুরস্কার এবং পাক-ভারত যুদ্ধ সম্পর্কে লেখা সাংবাদিকতাসুলভ রচনা-সংকলন রণাঙ্গন (১৯৬৬)-এর জন্য সিতারা-ই-ইমতিয়াজ উপাধি লাভ করেন। ১৯৬৭-৬৮ সালে ঢাকা বিশ্ববিদ্যালয়ের উদ্যোগে বাংলা বর্ণমালা ও বানান-পদ্ধতির সংস্কার প্রচেষ্টার প্রতিরোধের উদ্দেশ্যে প্রকাশিত একটি রিপোর্টে প্রবন্ধ লেখেন এবং পরবর্তীতে এ বিষয়ক বিতর্কে সক্রিয় অংশ নেন।'

In [5]:
df

,question,answers,context
0,ঔপন্যাসিক ও গল্প...,কখনো আসেনি,জহির রায়হান বাং...
1,বিশ্বের প্রথম চল...,লুমিয়ের ভ্রাতৃদ...,চলচ্চিত্রের ইতিহ...
2,গঠন ও প্রচলন নীত...,তিন,কম্পিউটারের গঠন ...
3,দেবশ্রী রায়ের ডা...,চুমকি,দেবশ্রী রায় এর ...
4,প্রিন্স দ্বারকান...,রামলোচনে,রবার্ট গুটলার ফা...
...,...,...,...
1246,মানুষ কোন গোত্রে...,হোমিনিনি,মানুষ বর্তমান পৃ...
1247,অপ্সরা শব্দটির উ...,সংস্কৃত শব্দ অপ্,সংস্কৃত শব্দ অপ্...
1248,ভারতীয় বিমানবাহ...,১৯৩২ সালের ৮ সেপ...,১৯৩২ সালের ৮ সেপ...
1249,পুরুষদের ক্ষেত্র...,১০-১৭ বছর,যদিও বয়ঃসন্ধি শ...
